### 문제 6-1 : 조건부 분기가 있는 메뉴 추천 시스템 ( LangGraph 사용하기)

In [ ]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()

llm = ChatGroq(
    model="llama3-70b-8192",
    temperature=0.3,
    api_key=os.getenv("GROQ_API_KEY")
)


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain.text_splitter import CharacterTextSplitter

# 1. 문서 로드 및 분할
loader = TextLoader("cafe_menu_data.txt", encoding="utf-8")
docs = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=300, chunk_overlap=50)
split_docs = text_splitter.split_documents(docs)

# 2. 벡터화 및 DB 생성
embeddings = OllamaEmbeddings(model="qwen2:0.5b")  # or your preferred local embedding
menu_db = FAISS.from_documents(split_docs, embeddings)


In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import MessagesState
from langchain_core.messages import HumanMessage, AIMessage
from typing import TypedDict, Literal

# 확장 상태 클래스
class CafeState(MessagesState):
    query_type: Literal["menu", "price", "recommend", "unknown"] = "unknown"
    search_results: list = []


In [ ]:
def classify_query(state: CafeState) -> str:
    last_msg = state["messages"][-1].content
    if "추천" in last_msg:
        return "recommend"
    elif "가격" in last_msg or "얼마" in last_msg:
        return "price"
    elif "메뉴" in last_msg or "카페라떼" in last_msg:
        return "menu"
    else:
        return "unknown"


In [ ]:
import re
from langchain_core.documents import Document

def extract_menu_info(doc: Document) -> dict:
    content = doc.page_content
    name = doc.metadata.get("menu_name", "Unknown")
    price = re.search(r"₩[\d,]+", content)
    description = re.search(r"설명:\s*(.+?)(?:\n|$)", content, re.DOTALL)
    
    return {
        "name": name,
        "price": price.group(0) if price else "가격 정보 없음",
        "description": description.group(1).strip() if description else "설명 없음"
    }


In [ ]:
def handle_menu_query(state: CafeState):
    q = state["messages"][-1].content
    results = menu_db.similarity_search(q, k=3)
    return {**state, "query_type": "menu", "search_results": results}

def handle_price_query(state: CafeState):
    results = menu_db.similarity_search("메뉴 가격", k=3)
    return {**state, "query_type": "price", "search_results": results}

def handle_recommend_query(state: CafeState):
    q = state["messages"][-1].content
    results = menu_db.similarity_search(q, k=2)
    if not results:
        results = menu_db.similarity_search("인기 메뉴", k=3)
    return {**state, "query_type": "recommend", "search_results": results}

def handle_unknown_query(state: CafeState):
    return {**state, "query_type": "unknown", "search_results": []}


In [ ]:
def generate_response(state: CafeState):
    query = state["messages"][-1].content
    results = state["search_results"]
    query_type = state["query_type"]

    if results:
        infos = [extract_menu_info(doc) for doc in results]
        summary = "\n\n".join([f"- {i['name']}: {i['price']} / {i['description']}" for i in infos])
    else:
        summary = "죄송합니다. 관련 메뉴 정보를 찾지 못했습니다."

    system_reply = f"[{query_type.upper()} 응답]\n{summary}"
    return {
        "messages": state["messages"] + [AIMessage(content=system_reply)],
        "query_type": query_type,
        "search_results": results
    }


In [ ]:
workflow = StateGraph(CafeState)

# 노드 등록
workflow.add_node("classify", classify_query)
workflow.add_node("menu", handle_menu_query)
workflow.add_node("price", handle_price_query)
workflow.add_node("recommend", handle_recommend_query)
workflow.add_node("unknown", handle_unknown_query)
workflow.add_node("respond", generate_response)

# 흐름 정의
workflow.set_entry_point("classify")
workflow.add_conditional_edges("classify", lambda s: classify_query(s), {
    "menu": "menu",
    "price": "price",
    "recommend": "recommend",
    "unknown": "unknown"
})
workflow.add_edge("menu", "respond")
workflow.add_edge("price", "respond")
workflow.add_edge("recommend", "respond")
workflow.add_edge("unknown", "respond")
workflow.add_edge("respond", END)

graph = workflow.compile()


In [ ]:
inputs = {
    "messages": [HumanMessage(content="카페라떼랑 어울리는 메뉴 추천해줘")]
}

final_state = graph.invoke(inputs)
print("\n🤖 최종 응답:\n")
print(final_state["messages"][-1].content)
